# CRNN Models with Augmentation and Ensemble Method
# Baseline Model

In [0]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# setup
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import math
from google.colab import drive
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt

from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Permute, Bidirectional, BatchNormalization, Conv2D,  Dense, Dropout, ELU, Flatten, MaxPool2D, TimeDistributed
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.model_selection import GridSearchCV

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Project247/'

X_test = np.load(root_path+"X_test.npy")
y_test = np.load(root_path+"y_test.npy")
person_train_valid = np.load(root_path+"person_train_valid.npy")
X_train_valid = np.load(root_path+"X_train_valid.npy")
y_train_valid = np.load(root_path+"y_train_valid.npy")
person_test = np.load(root_path+"person_test.npy")

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

# test GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


TensorFlow 2.x selected.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)
Found GPU at: /device:GPU:0


In [3]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)


In [0]:
# split function for pices of data
def split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = False, use_cv = False, cv = 5, size = 400, step = 20, end = 1000, printSize = False):
  y_test_norm = y_test - np.min(y_test)
  y_train_valid_norm = y_train_valid - np.min(y_train_valid)
  # split train and validation 80:20 (front-rare)
  if use_cv:
    X_train_valid_split = np.array( np.array_split(X_train_valid, cv) )
    X_train = np.concatenate(X_train_valid_split[0:cv-1])
    X_valid = X_train_valid_split[cv-1]
    y_train_valid_split = np.array( np.array_split(y_train_valid_norm, cv) )
    y_train = np.concatenate(y_train_valid_split[0:cv-1])
    y_valid = y_train_valid_split[cv-1]
  else:
    X_train = X_train_valid
    X_valid = X_train_valid[0:2]
    y_train = y_train_valid_norm
    y_valid = y_train_valid_norm[0:2]
  

  # make time be the index (1000, 22, 443)
  X_test_pcs_rsp = X_test.transpose(2,1,0)
  X_train_pcs_rsp = X_train.transpose(2,1,0)
  X_valid_pcs_rsp = X_valid.transpose(2,1,0)
  times = math.ceil( (end-size) /step )

  # split timestamp into small pieces (45, 100, 22, 443)
  if test_split:
    X_test_pcs_rsp = np.array([X_test_pcs_rsp[i : i + size] for i in range(0, end-size, step)])
  else:
    X_test_pcs_rsp = X_test_pcs_rsp[0 : size]
  X_train_pcs_rsp =  np.array([X_train_pcs_rsp[i : i + size] for i in range(0, end-size, step)])
  X_valid_pcs_rsp =  np.array([X_valid_pcs_rsp[i : i + size] for i in range(0, end-size, step)])
  
  # make samples be the index (45, 443, 22, 100)
  if test_split:
    X_test_pcs_rsp = X_test_pcs_rsp.transpose(0,3,2,1)
  else:
    X_test_pcs_rsp = X_test_pcs_rsp.transpose(2,1,0)
  X_train_pcs_rsp = X_train_pcs_rsp.transpose(0,3,2,1)
  X_valid_pcs_rsp = X_valid_pcs_rsp.transpose(0,3,2,1)
  
  # combine all samples together (45*443, 22, 100)
  if test_split:
    X_test_pcs_rsp = np.concatenate(X_test_pcs_rsp, axis=0)
  X_train_pcs_rsp = np.concatenate(X_train_pcs_rsp, axis=0)
  X_valid_pcs_rsp = np.concatenate(X_valid_pcs_rsp, axis=0)
  
  # repeat n times of the whole y array
  if test_split:
    y_test_norm_pcs = np.tile(y_test_norm, times)
  else:
    y_test_norm_pcs = y_test_norm
  y_train_norm_pcs = np.tile(y_train, times)
  y_valid_norm_pcs = np.tile(y_valid, times)
  if printSize:
      print(X_test_pcs_rsp.shape)
      print(X_train_pcs_rsp.shape)
      print(X_valid_pcs_rsp.shape)
      print(len(y_test_norm_pcs))
      print(len(y_train_norm_pcs))
      print(len(y_valid_norm_pcs))
  y_test_norm_pcs_b = to_categorical(y_test_norm_pcs)
  y_train_norm_pcs_b = to_categorical(y_train_norm_pcs)
  y_valid_norm_pcs_b = to_categorical(y_valid_norm_pcs)
  return X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs_b, y_valid_norm_pcs_b, y_test_norm_pcs_b

In [0]:
def rnn(init= GlorotUniform(seed=42),size=1000,l1_l=0.,l2_l=0.,optim='Adam',lr_r=0.001,decay_r=0.001):

  model = tf.keras.Sequential()
  model.add(layers.Input(shape=(size,22)))

  # reduce channels
  model.add(layers.Dense(16, kernel_regularizer=L1L2(l1=l1_l, l2=l2_l)))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  model.add(Permute((2, 1)))

  # reduce length
  model.add(layers.Dense(256, kernel_regularizer=L1L2(l1=l1_l, l2=l2_l)))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())

  # RNN
  model.add(Bidirectional(CuDNNLSTM(64, kernel_initializer=init, return_sequences=True)))
  
  # FC
  model.add(layers.Flatten())
  model.add(layers.Dense(64, kernel_regularizer=L1L2(l1=l1_l, l2=l2_l)))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  model.add(Dropout(0.4))

  model.add(layers.Dense(32, kernel_regularizer=L1L2(l1=l1_l, l2=l2_l)))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  model.add(Dropout(0.4))
  
  model.add(Dense(4, kernel_regularizer=L1L2(l1=l1_l, l2=l2_l), activation='softmax'))
  if optim == 'Adam':
      model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr_r, epsilon=1e-8, decay=decay_r), metrics=[categorical_accuracy])
  elif optim == 'RMSprop':
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=lr_r, rho=0.9), metrics=[categorical_accuracy])
  elif optim == 'Nesterov':
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=lr_r, decay=1e-6, momentum=0.9, nesterov=True), metrics=[categorical_accuracy])
    
  return model

Without Data Augmentation

In [87]:
RNNmodel = rnn(size=900)
RNNmodel.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_175 (Dense)            (None, 900, 16)           368       
_________________________________________________________________
batch_normalization_130 (Bat (None, 900, 16)           64        
_________________________________________________________________
re_lu_129 (ReLU)             (None, 900, 16)           0         
_________________________________________________________________
permute_36 (Permute)         (None, 16, 900)           0         
_________________________________________________________________
dense_176 (Dense)            (None, 16, 256)           230656    
_________________________________________________________________
batch_normalization_131 (Bat (None, 16, 256)           1024      
_________________________________________________________________
re_lu_130 (ReLU)             (None, 16, 256)         

In [88]:
X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = 900, step = 100, end = 1000, printSize = True)

X_test_pcs_reshaped = X_test_pcs_rsp.transpose(0,2,1)
X_train_pcs_reshaped = X_train_pcs_rsp.transpose(0,2,1)
X_valid_pcs_reshaped = X_valid_pcs_rsp.transpose(0,2,1)


with tf.device('/device:GPU:0'):
  # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
  history = RNNmodel.fit(X_train_pcs_reshaped, y_train_norm_pcs, epochs=20, batch_size=64, verbose=1, validation_data=(X_valid_pcs_reshaped, y_valid_norm_pcs))
# train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
val_score = history.history['val_categorical_accuracy'][-1]

(443, 22, 900)
(1692, 22, 900)
(423, 22, 900)
443
1692
423
Train on 1692 samples, validate on 423 samples
Epoch 1/20
1692/1692 [==============================] - 3s 2ms/sample - loss: 1.7831 - categorical_accuracy: 0.2595 - val_loss: 1.4772 - val_categorical_accuracy: 0.2411
Epoch 2/20
1692/1692 [==============================] - 0s 252us/sample - loss: 1.4394 - categorical_accuracy: 0.3528 - val_loss: 1.4852 - val_categorical_accuracy: 0.2766
Epoch 3/20
1692/1692 [==============================] - 0s 267us/sample - loss: 1.2309 - categorical_accuracy: 0.4616 - val_loss: 1.4511 - val_categorical_accuracy: 0.2695
Epoch 4/20
1692/1692 [==============================] - 0s 271us/sample - loss: 1.0817 - categorical_accuracy: 0.5366 - val_loss: 1.3886 - val_categorical_accuracy: 0.3357
Epoch 5/20
1692/1692 [==============================] - 0s 259us/sample - loss: 0.9080 - categorical_accuracy: 0.6413 - val_loss: 1.3897 - val_categorical_accuracy: 0.3499
Epoch 6/20
1692/1692 [==============

In [89]:
test_acc = RNNmodel.evaluate(X_test_pcs_reshaped,  y_test_norm_pcs, verbose=2)
# print('reg name',reg_name,'reg value',reg_value,'opt name',opt_name,'batch size',batch_size,'epochs',epochs)
print('test_acc',test_acc)

443/443 - 0s - loss: 2.4474 - categorical_accuracy: 0.3160
test_acc [2.447411144829227, 0.31602708]


With Data Augmentation

In [98]:
RNNmodel = rnn(size=900)
# 4x augmentation
X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = 900, step = 10, end = 940, printSize = True)

X_test_pcs_reshaped = X_test_pcs_rsp.transpose(0,2,1)
X_train_pcs_reshaped = X_train_pcs_rsp.transpose(0,2,1)
X_valid_pcs_reshaped = X_valid_pcs_rsp.transpose(0,2,1)

with tf.device('/device:GPU:0'):
  # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
  history = RNNmodel.fit(X_train_pcs_reshaped, y_train_norm_pcs, epochs=20, batch_size=64, verbose=1, validation_data=(X_valid_pcs_reshaped, y_valid_norm_pcs))
# train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
val_score = history.history['val_categorical_accuracy'][-1]

(1772, 22, 900)
(6768, 22, 900)
(1692, 22, 900)
1772
6768
1692
Train on 6768 samples, validate on 1692 samples
Epoch 1/20
6768/6768 [==============================] - 4s 648us/sample - loss: 1.5599 - categorical_accuracy: 0.2946 - val_loss: 1.4873 - val_categorical_accuracy: 0.2937
Epoch 2/20
6768/6768 [==============================] - 2s 253us/sample - loss: 1.3109 - categorical_accuracy: 0.3985 - val_loss: 1.3014 - val_categorical_accuracy: 0.3901
Epoch 3/20
6768/6768 [==============================] - 2s 252us/sample - loss: 1.1425 - categorical_accuracy: 0.4941 - val_loss: 1.2472 - val_categorical_accuracy: 0.4149
Epoch 4/20
6768/6768 [==============================] - 2s 250us/sample - loss: 0.9813 - categorical_accuracy: 0.5977 - val_loss: 1.2080 - val_categorical_accuracy: 0.4793
Epoch 5/20
6768/6768 [==============================] - 2s 252us/sample - loss: 0.8291 - categorical_accuracy: 0.6742 - val_loss: 1.4135 - val_categorical_accuracy: 0.4054
Epoch 6/20
6768/6768 [=======

In [99]:
test_acc = RNNmodel.evaluate(X_test_pcs_reshaped,  y_test_norm_pcs, verbose=2)
# print('reg name',reg_name,'reg value',reg_value,'opt name',opt_name,'batch size',batch_size,'epochs',epochs)
print('test_acc',test_acc)

1772/1772 - 1s - loss: 2.3697 - categorical_accuracy: 0.4633
test_acc [2.369718680801564, 0.4633183]


# Determine the Optimal Window Size
Without Data Augmentation

In [92]:
for time in range(100,1000,100):
  size = time
  step = 1000-size
  end = 1000
  X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)
  
  X_test_pcs_reshaped = X_test_pcs_rsp.transpose(0,2,1)
  X_train_pcs_reshaped = X_train_pcs_rsp.transpose(0,2,1)
  X_valid_pcs_reshaped = X_valid_pcs_rsp.transpose(0,2,1)

  RNNmodel = rnn(size=time)
  with tf.device('/device:GPU:0'):
    # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
    history = RNNmodel.fit(X_train_pcs_reshaped, y_train_norm_pcs, epochs=20, batch_size=64, verbose=0, validation_data=(X_valid_pcs_reshaped, y_valid_norm_pcs))
  # train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
  val_acc = history.history['val_categorical_accuracy'][-1]
  train_acc = history.history['categorical_accuracy'][-1]
  print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)

time window: 100 train_acc 0.928487 val_acc 0.3711584
time window: 200 train_acc 0.964539 val_acc 0.38770685
time window: 300 train_acc 0.97635937 val_acc 0.32624114
time window: 400 train_acc 0.9858156 val_acc 0.39716312
time window: 500 train_acc 0.9934988 val_acc 0.356974
time window: 600 train_acc 0.9946808 val_acc 0.38061467
time window: 700 train_acc 0.9934988 val_acc 0.38297874
time window: 800 train_acc 0.9958629 val_acc 0.3356974
time window: 900 train_acc 0.9946808 val_acc 0.30260047


With Data Augmentation

In [100]:
for time in range(100,1000,100):
  size = time
  step = 10
  end = int(size + step*4)    # gives 4x
  X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)

  X_test_pcs_reshaped = X_test_pcs_rsp.transpose(0,2,1)
  X_train_pcs_reshaped = X_train_pcs_rsp.transpose(0,2,1)
  X_valid_pcs_reshaped = X_valid_pcs_rsp.transpose(0,2,1)

  RNNmodel = rnn(size=time)
  with tf.device('/device:GPU:0'):
    # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
    history = RNNmodel.fit(X_train_pcs_reshaped, y_train_norm_pcs, epochs=20, batch_size=64, verbose=0, validation_data=(X_valid_pcs_reshaped, y_valid_norm_pcs))
  # train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
  val_acc = history.history['val_categorical_accuracy'][-1]
  train_acc = history.history['categorical_accuracy'][-1]
  print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)

time window: 100 train_acc 0.89612883 val_acc 0.41312057
time window: 200 train_acc 0.9571513 val_acc 0.46513003
time window: 300 train_acc 0.9762116 val_acc 0.5106383
time window: 400 train_acc 0.9824173 val_acc 0.49881798
time window: 500 train_acc 0.989805 val_acc 0.5395981
time window: 600 train_acc 0.98877066 val_acc 0.53014183
time window: 700 train_acc 0.9926123 val_acc 0.49527186
time window: 800 train_acc 0.99276006 val_acc 0.5065012
time window: 900 train_acc 0.990987 val_acc 0.47990543


# Grid Search for Optimal Hyperparameters
We pick time window sizes of 500 and 600

In [0]:
# For time window of 500
epoch_list = [20]
batch_list = [48]
opt_list = ['Adam','Nesterov','RMSprop']
reg_name_list = ['l2','l1','None']
reg_list = [0.01, 0.001]
step_list = [0.001]
decay_list = [0.01, 0.001]
size_list = [500]
total_iter = (len(epoch_list)*len(batch_list)*len(opt_list)*len(reg_name_list)*len(reg_list)*len(step_list)*len(decay_list)*len(size_list) - 
              len(epoch_list)*len(batch_list)*len(opt_list)*len(step_list)*len(decay_list)*len(size_list))

In [104]:
acc_list = []
param_list = []
iter=0

for size in size_list:
  step = 10
  end = int(size + step*4)   # gives 4x
  X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)
  # preprocess data
  X_test_pcs_reshaped = X_test_pcs_rsp.transpose(0,2,1)
  X_train_pcs_reshaped = X_train_pcs_rsp.transpose(0,2,1)
  X_valid_pcs_reshaped = X_valid_pcs_rsp.transpose(0,2,1)
  for reg_name in reg_name_list:
    if reg_name == 'l1':
      l1_l_list = reg_list
      l2_l_list = [0]
    elif reg_name == 'l2':
      l2_l_list = reg_list
      l1_l_list = [0]
    else:
      l1_l_list = [0]
      l2_l_list = [0]
    for l1_l in l1_l_list:
      for l2_l in l2_l_list:
        for opt in opt_list:
          for step in step_list:
            for decay in decay_list:
              for batch in batch_list:
                for epoch in epoch_list:
                  iter += 1
                  print('Iteration',iter,'of',total_iter)
                  RNNmodel = rnn(size=size,l1_l=l1_l,l2_l=l2_l,optim=opt,lr_r=step,decay_r=decay)
                  with tf.device('/device:GPU:0'):
                    # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
                    history = RNNmodel.fit(X_train_pcs_reshaped, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=0, validation_data=(X_valid_pcs_reshaped, y_valid_norm_pcs))
                  # train_score = RNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
                  val_acc = history.history['val_categorical_accuracy'][-1]
                  train_acc = history.history['categorical_accuracy'][-1]
                  print('size',size,'reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch)
                  print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)  # test on validation data
                  param_list.append(['size',size,'reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch])
                  acc_list.append(val_acc)


Iteration 1 of 30
size 500 reg name l2 reg value 0 0.01 opt name Adam step 0.001 decay 0.01 batch size 48 epochs 20
time window: 500 train_acc 0.88327426 val_acc 0.4787234
Iteration 2 of 30
size 500 reg name l2 reg value 0 0.01 opt name Adam step 0.001 decay 0.001 batch size 48 epochs 20
time window: 500 train_acc 0.974143 val_acc 0.55082744
Iteration 3 of 30
size 500 reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.01 batch size 48 epochs 20
time window: 500 train_acc 0.7802896 val_acc 0.48640662
Iteration 4 of 30
size 500 reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 20
time window: 500 train_acc 0.81028366 val_acc 0.46453902
Iteration 5 of 30
size 500 reg name l2 reg value 0 0.01 opt name RMSprop step 0.001 decay 0.01 batch size 48 epochs 20
time window: 500 train_acc 0.9159279 val_acc 0.5596927
Iteration 6 of 30
size 500 reg name l2 reg value 0 0.01 opt name RMSprop step 0.001 decay 0.001 batch size 48 epochs 20
time win

In [105]:
# Output the top # of parameters
top_num = 5
acc_sorted, para_sorted = zip(*sorted(zip(acc_list, param_list),reverse=True))
for j in range(top_num):
  print('#',j+1,'accuracy')
  print(para_sorted[j])
  print(acc_sorted[j])

# 1 accuracy
['size', 500, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'RMSprop', 'step', 0.001, 'decay', 0.01, 'batch size', 48, 'epochs', 20]
0.5809693
# 2 accuracy
['size', 500, 'reg name', 'l2', 'reg value', 0, 0.001, 'opt name', 'RMSprop', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 20]
0.57978725
# 3 accuracy
['size', 500, 'reg name', 'l2', 'reg value', 0, 0.001, 'opt name', 'RMSprop', 'step', 0.001, 'decay', 0.01, 'batch size', 48, 'epochs', 20]
0.57742316
# 4 accuracy
['size', 500, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Adam', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 20]
0.56678486
# 5 accuracy
['size', 500, 'reg name', 'l2', 'reg value', 0, 0.01, 'opt name', 'RMSprop', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 20]
0.5620567


In [0]:
# For time window of 600
epoch_list = [20]
batch_list = [48]
opt_list = ['Adam','Nesterov','RMSprop']
reg_name_list = ['l2','l1','None']
reg_list = [0.01, 0.001]
step_list = [0.001]
decay_list = [0.01, 0.001]
size_list = [600]
total_iter = (len(epoch_list)*len(batch_list)*len(opt_list)*len(reg_name_list)*len(reg_list)*len(step_list)*len(decay_list)*len(size_list) - 
              len(epoch_list)*len(batch_list)*len(opt_list)*len(step_list)*len(decay_list)*len(size_list))

In [107]:
acc_list = []
param_list = []
iter=0

for size in size_list:
  step = 10
  end = int(size + step*4)   # gives 4x
  X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)
  # preprocess data
  X_test_pcs_reshaped = X_test_pcs_rsp.transpose(0,2,1)
  X_train_pcs_reshaped = X_train_pcs_rsp.transpose(0,2,1)
  X_valid_pcs_reshaped = X_valid_pcs_rsp.transpose(0,2,1)
  for reg_name in reg_name_list:
    if reg_name == 'l1':
      l1_l_list = reg_list
      l2_l_list = [0]
    elif reg_name == 'l2':
      l2_l_list = reg_list
      l1_l_list = [0]
    else:
      l1_l_list = [0]
      l2_l_list = [0]
    for l1_l in l1_l_list:
      for l2_l in l2_l_list:
        for opt in opt_list:
          for step in step_list:
            for decay in decay_list:
              for batch in batch_list:
                for epoch in epoch_list:
                  iter += 1
                  print('Iteration',iter,'of',total_iter)
                  RNNmodel = rnn(size=size,l1_l=l1_l,l2_l=l2_l,optim=opt,lr_r=step,decay_r=decay)
                  with tf.device('/device:GPU:0'):
                    # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
                    history = RNNmodel.fit(X_train_pcs_reshaped, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=0, validation_data=(X_valid_pcs_reshaped, y_valid_norm_pcs))
                  # train_score = RNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
                  val_acc = history.history['val_categorical_accuracy'][-1]
                  train_acc = history.history['categorical_accuracy'][-1]
                  print('size',size,'reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch)
                  print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)  # test on validation data
                  param_list.append(['size',size,'reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch])
                  acc_list.append(val_acc)


Iteration 1 of 30
size 600 reg name l2 reg value 0 0.01 opt name Adam step 0.001 decay 0.01 batch size 48 epochs 20
time window: 600 train_acc 0.9354314 val_acc 0.4710402
Iteration 2 of 30
size 600 reg name l2 reg value 0 0.01 opt name Adam step 0.001 decay 0.001 batch size 48 epochs 20
time window: 600 train_acc 0.97118795 val_acc 0.5070922
Iteration 3 of 30
size 600 reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.01 batch size 48 epochs 20
time window: 600 train_acc 0.785461 val_acc 0.43144208
Iteration 4 of 30
size 600 reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 20
time window: 600 train_acc 0.7915189 val_acc 0.4503546
Iteration 5 of 30
size 600 reg name l2 reg value 0 0.01 opt name RMSprop step 0.001 decay 0.01 batch size 48 epochs 20
time window: 600 train_acc 0.9159279 val_acc 0.5254137
Iteration 6 of 30
size 600 reg name l2 reg value 0 0.01 opt name RMSprop step 0.001 decay 0.001 batch size 48 epochs 20
time window

In [108]:
# Output the top # of parameters
top_num = 5
acc_sorted, para_sorted = zip(*sorted(zip(acc_list, param_list),reverse=True))
for j in range(top_num):
  print('#',j+1,'accuracy')
  print(para_sorted[j])
  print(acc_sorted[j])

# 1 accuracy
['size', 600, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'RMSprop', 'step', 0.001, 'decay', 0.01, 'batch size', 48, 'epochs', 20]
0.56264776
# 2 accuracy
['size', 600, 'reg name', 'None', 'reg value', 0, 0, 'opt name', 'RMSprop', 'step', 0.001, 'decay', 0.01, 'batch size', 48, 'epochs', 20]
0.5608747
# 3 accuracy
['size', 600, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Adam', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 20]
0.55910164
# 4 accuracy
['size', 600, 'reg name', 'l2', 'reg value', 0, 0.001, 'opt name', 'RMSprop', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 20]
0.55260044
# 5 accuracy
['size', 600, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'RMSprop', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 20]
0.5437352


# Run the Optimal Model and Evaluate the Testing Accuracy
We use higher volume of data augmentation, from 4x to 8x.

In [128]:
# ['size', 500, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'RMSprop', 'step', 0.001, 'decay', 0.01, 'batch size', 48, 'epochs', 20]

size = 500
step = 10
end = int(size + step*8)    # gives 8x

X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
  split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = False, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = True)

# preprocess data
X_test_pcs_reshaped = X_test_pcs_rsp.transpose(0,2,1)
X_train_pcs_reshaped = X_train_pcs_rsp.transpose(0,2,1)
X_valid_pcs_reshaped = X_valid_pcs_rsp.transpose(0,2,1)
reg_name = 'l1'
l1_l = 0.001
l2_l = 0
opt = 'RMSprop'
step = 0.001
decay = 0.01
batch = 48
epoch = 20

RNNmodel = rnn(size=size,l1_l=l1_l,l2_l=l2_l,optim=opt,lr_r=step,decay_r=decay)
with tf.device('/device:GPU:0'):
  # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
  history = RNNmodel.fit(X_train_pcs_reshaped, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=1, validation_data=(X_valid_pcs_reshaped, y_valid_norm_pcs))
# train_score = CRNNopt.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
val_acc = history.history['val_categorical_accuracy'][-1]
train_acc = history.history['categorical_accuracy'][-1]
print('reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch)
print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)  # test on validation data

(443, 22, 500)
(13536, 22, 500)
(3384, 22, 500)
443
13536
3384
Train on 13536 samples, validate on 3384 samples
Epoch 1/20
13536/13536 [==============================] - 7s 512us/sample - loss: 7.0974 - categorical_accuracy: 0.3479 - val_loss: 3.5129 - val_categorical_accuracy: 0.4122
Epoch 2/20
13536/13536 [==============================] - 4s 279us/sample - loss: 2.5902 - categorical_accuracy: 0.4720 - val_loss: 2.2518 - val_categorical_accuracy: 0.4764
Epoch 3/20
13536/13536 [==============================] - 4s 279us/sample - loss: 2.1428 - categorical_accuracy: 0.5188 - val_loss: 2.1644 - val_categorical_accuracy: 0.5053
Epoch 4/20
13536/13536 [==============================] - 4s 272us/sample - loss: 2.0322 - categorical_accuracy: 0.5482 - val_loss: 2.0713 - val_categorical_accuracy: 0.5201
Epoch 5/20
13536/13536 [==============================] - 4s 279us/sample - loss: 1.9903 - categorical_accuracy: 0.5703 - val_loss: 2.0615 - val_categorical_accuracy: 0.5361
Epoch 6/20
13536/1

In [129]:
test_acc = RNNmodel.evaluate(X_test_pcs_reshaped,  y_test_norm_pcs, verbose=2)
# print('reg name',reg_name,'reg value',reg_value,'opt name',opt_name,'batch size',batch_size,'epochs',epochs)
print('test_acc',test_acc)

443/443 - 0s - loss: 2.7556 - categorical_accuracy: 0.5485
test_acc [2.755596230853492, 0.5485327]


Evaluate on individual subjects

In [130]:
predictions = RNNmodel.predict(X_test_pcs_reshaped)
print(predictions.shape)

(443, 4)


In [131]:
# Accuracy for each subject
for i in range(9): 
  print('Subject',i)
  index_test = list(filter(lambda x: person_test[x] == i, range(len(person_test))))
  pred_i = predictions[index_test]
  true_i = y_test_norm_pcs[index_test]
  pred_label = np.array( np.argmax(pred_i, axis=1) )
  true_label = np.array( np.argmax(true_i, axis=1) )
  acc_final = np.sum(pred_label == true_label) / true_label.shape[0]
  print('Pred acc',acc_final)

Subject 0
Pred acc 0.52
Subject 1
Pred acc 0.46
Subject 2
Pred acc 0.54
Subject 3
Pred acc 0.58
Subject 4
Pred acc 0.7021276595744681
Subject 5
Pred acc 0.5102040816326531
Subject 6
Pred acc 0.52
Subject 7
Pred acc 0.48
Subject 8
Pred acc 0.6382978723404256


In [0]:
from google.colab import files
np.savetxt("bestRNN_predict.csv", predictions, delimiter=",")
files.download('bestRNN_predict.csv')

Train on individual subjects

In [0]:
def pickUpOne(X_train_valid, X_test, y_train_valid, y_test, person_train_valid, person_test, index=0):
  index_train_valid = list(filter(lambda x: person_train_valid[x] == index, range(len(person_train_valid)))) 
  index_test = list(filter(lambda x: person_test[x] == index, range(len(person_test)))) 
  X_train_valid_first = X_train_valid[index_train_valid]
  y_train_valid_first = y_train_valid[index_train_valid]
  X_test_first = X_test[index_test]
  y_test_first = y_test[index_test]
  return X_train_valid_first, X_test_first, y_train_valid_first, y_test_first

In [135]:
for index in range(9):
  print('Working on subject',index)
  X_train_valid_sub, X_test_sub, y_train_valid_sub, y_test_sub = pickUpOne(X_train_valid, X_test, y_train_valid, y_test, person_train_valid, person_test, index)

  pred_list_all = []
  pred_list_all_sub = []
  # ['size', 900, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]

  size = 500
  step = 10
  end = int(size + step*8)    # gives 8x

  X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid_sub, X_test_sub, y_train_valid_sub, y_test_sub, test_split = False, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)

  # preprocess data
  X_test_pcs_reshaped = X_test_pcs_rsp.transpose(0,2,1)
  X_train_pcs_reshaped = X_train_pcs_rsp.transpose(0,2,1)
  X_valid_pcs_reshaped = X_valid_pcs_rsp.transpose(0,2,1)
  
  reg_name = 'l1'
  l1_l = 0.001
  l2_l = 0
  opt = 'RMSprop'
  step = 0.001
  decay = 0.01
  batch = 48
  epoch = 20

  RNNmodel = rnn(size=size,l1_l=l1_l,l2_l=l2_l,optim=opt,lr_r=step,decay_r=decay)
  with tf.device('/device:GPU:0'):
    # validation_split=0.2 or validation_data=(X_valid_pcs_reshaped, y_valid_norm_pcs)
    history = RNNmodel.fit(X_train_pcs_reshaped, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=0, validation_data=(X_valid_pcs_reshaped, y_valid_norm_pcs))
  test_loss, test_acc = RNNmodel.evaluate(X_test_pcs_reshaped,  y_test_norm_pcs, verbose=2)

  print('Subject',index,'test_acc',test_acc)


Working on subject 0
50/50 - 0s - loss: 2.8431 - categorical_accuracy: 0.5400
Subject 0 test_acc 0.54
Working on subject 1
50/50 - 0s - loss: 4.4208 - categorical_accuracy: 0.3000
Subject 1 test_acc 0.3
Working on subject 2
50/50 - 0s - loss: 8.7765 - categorical_accuracy: 0.3000
Subject 2 test_acc 0.3
Working on subject 3
50/50 - 0s - loss: 3.3305 - categorical_accuracy: 0.3600
Subject 3 test_acc 0.36
Working on subject 4
47/47 - 0s - loss: 2.5619 - categorical_accuracy: 0.6596
Subject 4 test_acc 0.65957445
Working on subject 5
49/49 - 0s - loss: 4.5802 - categorical_accuracy: 0.4286
Subject 5 test_acc 0.42857143
Working on subject 6
50/50 - 0s - loss: 2.2847 - categorical_accuracy: 0.6400
Subject 6 test_acc 0.64
Working on subject 7
50/50 - 0s - loss: 3.4266 - categorical_accuracy: 0.4400
Subject 7 test_acc 0.44
Working on subject 8
47/47 - 0s - loss: 1.9273 - categorical_accuracy: 0.7872
Subject 8 test_acc 0.78723407
